# Context Managers
This tutorial introduces contexts. A _context_ is a wrapper tha modifies the behavior of a block of code. This tutorial illustrates how to define a context using the `@contextmanager` decorator and how to invoke one using python's `with` statement. It does not describe the lower-level mechanisms upon which the decorator is based. That documentation is in the python manual, here: [`with` statement context managers](https://docs.python.org/3/reference/datamodel.html#context-managers)

# The Canonical Example
We begin with an example illustrating what a context is. This example is so ubiquitous in the python literature that virtually every explanation of contexts uses it:

Suppose we wish to read and process the contents of a file. We first must _open_ the file, which is essentially asking the operating system for a file descriptor, and, when we are done with the file, we have to _close_ it; otherwise, the operating system thinks we're still using it. We have to write code that closes the file even if an exception is raised that causes a non-local exit from our code. That is typically handled using a template like the one below:

In [1]:
f = open('my_file.txt')  # Get file descriptor
try:
    print(f.readline(), end = '') # operate on the file's contents
    print(f.readline(), end = '') # operate on the file's contents
    print(f.readline(), end = '') # operate on the file's contents

finally:
    f.close() # return the descriptor back to the operating system.

Hello, World!
It's a beautiful today to learn python.
Today I'm learning about context managers.


Of the eight lines above, four of them constitute the context for file handling, irrespective of what specific code goes on lines 3-5, which could be any code at all that operates on the file:

- Open the file and assign the resulting file descriptor object to a variable.
- Establish a `try/except` block
- Make sure the file gets closed regardless of any non-local exit by putting the `close()` operation in a `finally` block.

Requiring the coder to write these four context lines everywhere they wish to process a file is somewhat tedious, and also somewhat error prone, because the coder has to remember the correct ritual for guaranteeing the closing of `f`. Furthermore, if the correct ritual changes one day, then altering the code to reflect the new style involves hunting for every occurence of this template within the code and changing it manually--even more tedious, and even more error-prone. 

A better solution is to define in a single place the template that represents the context. Once the context is defined, it can be invoked whenever it is needed. (Note: we haven't yet seen how `open()` is defined, and we won't see that, because python defines `open()` as part of the language. We'll use other examples below to illustrate how a context gets defined.) Below is what that invocation of the `open()` context looks like in python:

In [2]:
with open('my_file.txt') as f:
    print(f.readline(), end = '') # operate on the file's contents
    print(f.readline(), end = '') # operate on the file's contents
    print(f.readline(), end = '') # operate on the file's contents

    

Hello, World!
It's a beautiful today to learn python.
Today I'm learning about context managers.


In the above example code, the file `my_file.txt` is opened for reading and a file descriptor is assigned to the variable `f`. The `with` statement invokes the context object that is returned by `open('my_file.txt')`, then executes `print(f.readline())` inside of that context, which guarantees that the file descriptor `f` will be closed after the body of the `with` is exited, even if a non-local exit occurs. It is seemingly as though the above two lines of code are replaced with all five lines of [1], with line [2]:2 sandwiched into the middle of the context as line [1]:3. (This is not really what happens, and the context established by `with` does _not_ have access to the local variables of the caller.)

Notice the absence of explicit `try` and `except` language in the above example. All of that is implicit in the context established by the `with` statement.

Because the above example is so common, python implements the `open` context as part of the language. The above example works without needing to define anything else, and is the preferred way to open a file in python.

# A context is a code sandwich
A context is a way to make a "code sandwich" in which the middle of the sandwich can be any arbitrary code. In this analogy, the context is the two pieces of bread that wrap up the arbitrary code. One of the pieces of bread is the "enter" phase, and the other is the "exit" phase. These pieces of code are fixed and they define the "context".

Contexts and decorators play similar roles. They are both wrappers. A context wraps an arbitrary code block, while a decorator wraps an arbitrary function. 

A context doesn't need to involve a `try/except/finally` block, but typically it does, because the primary use case is establish some data structures, allocate resources, etc., in the "enter" phase and then ensure they get cleaned up in the "exit" phase.

# Defining Contexts
Now we are going to explore how to define contexts by introducing a sequence of progressively more complicated examples.

Remember that a context is a code sandwich that defines the pieces of bread as the context and inserts arbitrary code in the middle of the sandwich.  When we use the `@contextmanager` decorator to define a context, the "meat" of the sandwich is demarcated by a single `yield` statement. Below is a simple example of a context defined in this way. Note the `yield` statement on line 8 that marks the place where python should insert the meat of the sandwich. Because the presence of `yield` always turns a function into a generator, we note in passing that the `@contextmanager` decorator leverages the generator/iterator functionality of python to create the desired context.

NOTE: The `@contextmanager` decorator must be imported from module `contextlib`, as illustrated below.

In [3]:
# Simplest context
from contextlib import contextmanager

@contextmanager
def sandwiched_code():
    print("This is the enter code.")
    print("It consists of everything before yield.")
    yield
    print("This is the exit code.")
    print("It consists of everything after yield.")
    
with sandwiched_code():
    print("** This is the meat of the sandwich.")
    print("** You can put anything here.")
    print("** It gets wrapped by the context's enter and exit phases.")

This is the enter code.
It consists of everything before yield.
** This is the meat of the sandwich.
** You can put anything here.
** It gets wrapped by the context's enter and exit phases.
This is the exit code.
It consists of everything after yield.


A context definition should contain precisely one `yield` statement: no more, no less. The code below fails because it contains two `yield` statements.

In [4]:
# An incorrectly defined context.
# This context definition uses yield twice. It will break.
from contextlib import contextmanager

@contextmanager
def sandwiched_code2():
    print("This is the enter code.")
    print("It consists of everything before yield.")
    yield
    print("This is the exit code.")
    yield # This yield doesn't belong here! The code breaks when it gets here.
    print("It consists of everything after yield.")
    

In [5]:
# We expect this code to break.
with sandwiched_code2():
    print("This is the meat of the sandwich.")

This is the enter code.
It consists of everything before yield.
This is the meat of the sandwich.
This is the exit code.


RuntimeError: generator didn't stop

# A context need not have any "bread"
It is perfectly acceptable for a context to have no enter phase, or no exit phase, or neither. All it needs is a `yield` statement. An empty context is not particularly useful, but it is valid.

In [6]:
# Establish a do-nothing context
from contextlib import contextmanager

@contextmanager
def do_nothing():
    yield
    

with do_nothing():
    print("This works!")


This works!


# Contexts can take arguments
You can pass in arguments to a context. These can be used to parameterize the context.

In [7]:
# Establish a context for writing a letter to someone.
from contextlib import contextmanager

@contextmanager
def letter_writing(to_person, from_person):
    print("Dear,", to_person)
    yield
    print("Very Truly Yours,")
    print(from_person)

In [8]:
with letter_writing('Helen', 'Menelaus'):
    print("I miss you terribly!")

Dear, Helen
I miss you terribly!
Very Truly Yours,
Menelaus


# Contexts can yield values back to the caller
The calling code that uses a `with` statement to establish a context can receive values back from the context. The values received are whatever values are arguments to the `yield` statement within the context definition.

In [9]:
# Establish a context for writing a letter to someone.
# In this version, the letter is returned as a list of strings
# in the variable named "lines".
from contextlib import contextmanager

@contextmanager
def letter_writing(to_person, from_person):
    lines = []
    lines.append("Dear, " + to_person)
    yield lines
    lines.append("Very Truly Yours,")
    lines.append(from_person)

In [10]:
with letter_writing('Helen', 'Menelaus') as letter_lines:
    letter_lines.append("I miss you terribly!")
print('\n'.join(letter_lines))

Dear, Helen
I miss you terribly!
Very Truly Yours,
Menelaus


# Contexts can yield multiple values back to the caller
This should be obvious from what was said above, but here we give a motivated example and illustrate the syntax as well.

In [11]:
# Establish a context for writing a letter to someone.
# In this version, we take as input the relationship between
# the sender and receiver and yield style advice about whether
# the contents of the letter should be formal or informal.
from contextlib import contextmanager

@contextmanager
def letter_writing(to_person, from_person, relationship):
    # Determine style of address
    if relationship == 'spouse':
        style = 'informal'
    else:
        style = 'formal'
        
    lines = []
    lines.append("Dear, " + to_person)
    yield lines, style
    lines.append("Very Truly Yours,")
    lines.append(from_person)

In [12]:
def make_i_miss_you_letter(to_person, from_person, relationship):
    with letter_writing(to_person, from_person, relationship) as (letter_lines, style):
        if style == 'informal':
            letter_lines.append("I miss you terribly!")
        else:
            letter_lines.append("I count the hours until we are reunited.")
        print('\n'.join(letter_lines))       

make_i_miss_you_letter('Helen', 'Menelaus', 'spouse')
print()
make_i_miss_you_letter('Helen', 'Menelaus', 'girlfriend')

Dear, Helen
I miss you terribly!

Dear, Helen
I count the hours until we are reunited.


# Contexts generally use `try/except/finally` blocks
Contexts generally exist to ensure that all necessary resources are set up in advance of the "meat" of the sandwich, and then taken down properly afterwards, irrespective of whether or not a non-local exit occurred during the meat of the sandwich. For this reason, contexts usually have `try/except/finally` blocks. 

In [13]:
# In this example, we make sure to greet the person
# and say goodbye to them. We arrange to say goodbye
# even if some non-local exit occurs during the middle
# of our interaction with them.

from contextlib import contextmanager

@contextmanager
def greeting_and_farewell(to_person):
    print("Hello, " + to_person + '.')    
    try:
        yield
    finally: 
       print("Nice talking to you, " + to_person + ". See you soon!")

In [14]:
# We expect this example to break into the debugger, but not before
# it also says goodbye.
with greeting_and_farewell('John'):
    print("Would you be available to chat with me about python sometime?")
    x = 1 / 0 # Oops, an exception occurs here, causing a non-local exit.
              # But we still manage to say goodbye to John!


Hello, John.
Would you be available to chat with me about python sometime?
Nice talking to you, John. See you soon!


ZeroDivisionError: division by zero

# A Context doesn't have access to the local environment of its caller.
Below we define a context that references the free variable `person`. Even though `person` is a local variable
in the caller, the context has no special access to it. This proves that the context is not simply "inlined" where it is invoked. It really is more like a function call.

In [15]:
@contextmanager
def no_local_access():
    print('Hello, ' + person) # This reference to person won't resolve.
    yield
    
def foo():
    person = 'John'
    with no_local_access():
        pass
        
# The error below is expected. The context has no special access to the local variable 
# person in the caller.
foo()

NameError: name 'person' is not defined

# The body of a `with` statement has no access to the local environment of its context
Similarly, the body of a `with` statement cannot see the local variables established by the context (unless the context yields them back to the caller).


In [16]:
@contextmanager
def no_local_access():
    person = 'John'
    yield
    
def foo():
    with no_local_access():
        print("Hello, " + person) # This reference to person won't resolve.
        
# The error below is expected. The body of the with statement has no special
# access to the local variable person in the context.
foo()

NameError: name 'person' is not defined

# The body of a `with` statement _does_ have access to local variables outside the `with` statement.
As you would expect, the code below works just fine.

In [17]:
@contextmanager
def do_nothing():
    yield
    
def foo():
    person = 'John'
    with do_nothing():
        print("Hello, " + person) # This reference to person is fine.
        

foo()

Hello, John


# The "function" defined by @contextmanager returns a context object with an embedded generator
Remember that, because it contains a `yield` statement, the "function" that we decorate with `@contextmanager` is actually a generator. As such, it returns immediately when it is called. The decorator then wraps the generator in such a way as to make it an appropriate context object, which means that it defines `__enter__()` and `__exit__()` methods, and the `with` statement invokes those methods on the context object. We can see these pieces in the following example sequence. 

In [18]:
# Simplest context
from contextlib import contextmanager

@contextmanager
def sandwiched_code():
    print("This is the enter code.")
    print("It consists of everything before yield.")
    yield
    print("This is the exit code.")
    print("It consists of everything after yield.")
    

context = sandwiched_code()
context

In [19]:
context.__enter__

<bound method _GeneratorContextManager.__enter__ of <contextlib._GeneratorContextManager object at 0x10cc6d8d0>>

In [20]:
context.__exit__

<bound method _GeneratorContextManager.__exit__ of <contextlib._GeneratorContextManager object at 0x10cc6d8d0>>

In [21]:
# Here we see that the with statement works by actually
# being given a context object to invoke.
with context:
    print("This is the meat of the sandwich.")

This is the enter code.
It consists of everything before yield.
This is the meat of the sandwich.
This is the exit code.
It consists of everything after yield.


In [22]:
# This fails, because the context object contains
# an embedded generator, which we already exhausted above.
with context:
    print("This is the meat of the sandwich.")

RuntimeError: generator didn't yield

# Contexts can be methods
We can use `@contextmanager` to decorate a method as well as a function. The example below illustrates.

In [23]:
# In this example, we make sure to greet the person
# and say goodbye to them. We arrange to say goodbye
# even if some non-local exit occurs during the middle
# of our interaction with them.

from contextlib import contextmanager

class Salutation:
    """
    Represents a pair of phrases that are used
    to say hello and goodbye respectively. 
    
    Knows how to set up a context the begins with a hello
    and ends with a goodbye.
    """
    def __init__(self, hello, goodbye):
        self.hello = hello
        self.goodbye = goodbye

    @contextmanager
    def greeting_and_farewell(self, to_person):
        print(self.hello + ", " + to_person + '.')    
        try:
            yield
        finally: 
            print(self.goodbye + ", " + to_person + '.')
            
salutation = Salutation("Good Morning", "Nice talking to you")
with salutation.greeting_and_farewell('John'):
    print("How is life?")

Good Morning, John.
How is life?
Nice talking to you, John.


# A Final Example
The final example below is a little bit more elaborate, providing a motivated example for the use of contexts.

In [24]:
# This example defines a context that alters the built-in print() function so that
# it prefixes any print statement with the current time, restoring the original
# print() function when the context exits.

from contextlib import contextmanager
import datetime


@contextmanager
def print_prefixed_by_timestamp(date_time_format):
    """
    Prefix any print statement within our context by the current date and time,
    formatting it as specified by date_time_format.
    :param date_time_format: A string that specifies the format for the date and time,
                             whose syntax is dictated by the strftime() method of datetime.datime.
    :return: None
    """
    global print # This allows us to re-assign print to another function.

    old_print = print # Save the old print() so we can restore it later.

    # Define a new print function that prefixes the formatted time when printing.
    def new_print(s, *args, **kwds):
        """
        Print the string s, prefixed by the current time.
        :param s:
        :return: None
        """
        old_print(datetime.datetime.now().strftime(date_time_format), end = '--')
        old_print(s, *args, **kwds)


    print = new_print  # Bash the built-in print() to use our new print function
    try:
        yield              # yield the context: all print() calls done here will use new_print()
    finally:
        print = old_print  # Restore the original print function


In [25]:
with print_prefixed_by_timestamp('%Y/%m/%d %H:%M:%S'):
    print("Hello World")
    print("Nice to know what time it is.")
print("No more time-prefixing anymore.")

2018/04/24 08:06:54--Hello World
2018/04/24 08:06:54--Nice to know what time it is.
No more time-prefixing anymore.
